In [44]:
# https://github.com/chanzuckerberg/single-cell-curation/issues/516
# https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/4.0.0/schema.md#feature_length

import numpy as np
import os
import scanpy as sc
import subprocess
import pandas as pd

In [45]:
def validate(input_file, output_file):
    ''' 
    Input: h5ad file
    Output: h5ad file with additional var metadata fields
    
    '''
    validate_process = subprocess.run(['cellxgene-schema', 'validate', '--add-labels', input_file, output_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in validate_process.stdout.decode('utf-8').split('\n'):
        print(line)
    for line in validate_process.stderr.decode('utf-8').split('\n'):
        print(line)
        if 'is_valid=' in line:
            valid = line.split('=')[-1]
            print(valid)

In [43]:
def save(adata, filename):
    adata.write(filename)
    adata = sc.read_h5ad(filename)
    if adata.raw.var:
        print('adata.raw.var exists')
        if ('feature_length' not in adata.raw.var.columns) and ('feature_biotype' not in adata.raw.var.columns):
            print('feature_length and feature_biotype are absent - good CELLxGENE should annotate these two var fields')
    
        elif ('feature_biotype' in adata.raw.var.columns) and ('feature_length' not in adata.raw.var.columns):
            print('feature_biotype is present')
            print(adata.raw.var['feature_biotype'].info())
            print('-------------------')
    
        else: 
            print(adata.raw.var[['feature_biotype','feature_length']].info())
    
    else:
        print('adata.var exists')
        if ('feature_length' not in adata.var.columns) and ('feature_biotype' not in adata.var.columns):
            print('feature_length and feature_biotype are absent - good CELLxGENE should annotate these two var fields')
    
        elif ('feature_biotype' in adata.var.columns) and ('feature_length' not in adata.var.columns):
            print('feature_biotype is present')
            print(adata.var['feature_biotype'].info())
            print('-------------------')
    
        else: 
            print(adata.var[['feature_biotype','feature_length']].info())

        print('------------------')

In [46]:
def concat_expected_length_dfs(list_of_csvs):
    df_list = []
    for c in list_of_csvs:
        df = pd.read_csv(c,header=None)
        df_list.append(df)
    
    return pd.concat(df_list)

In [47]:
def check_var_metadata(output_file, ensg_id_to_check_list, list_of_csvs):
    ''' 
    Input: 1) h5ad validate output file, 2) list of ensembl gene ids to check, 3) list of csvs with expected gene lengths 
    Output: Dataframe of var metadata checks of both adata.raw.var and adata.var for feature_biotype and feature_length
    '''

    adata = sc.read_h5ad(output_file)
    gene_df = concat_expected_length_dfs(list_of_csvs)
    gene_df.set_index(0, inplace=True)
    result_dict = {}

    for e in ensg_id_to_check_list:
        result_list = []
        expected_length = gene_df.loc[e, 3]
        result_list.append(expected_length)
        if adata.raw.var:
            print('Checking adata.raw.var')
            print('---------------------------')
            if ('feature_biotype' in adata.raw.var.columns) and ('feature_length' in adata.raw.var.columns):
                print('feature_biotype and feature_length are present in raw.var fields')
                print('------------------------------------------------------------')
                print('Checking feature_length of: ', e)
                biotype = adata.raw.var.loc[e, 'feature_biotype']
                if biotype == 'gene':
                    print('biotype = gene')
                    result_list.append(biotype)
                    length = adata.raw.var.loc[e, 'feature_length']
                    result_list.append(length)
                    if length == expected_length:
                        print(f'feature_length {length} is the same as expected {expected_length}')
                        result_list.append(True)
                    else:
                        result_list.append(False)
                else:
                    print(f'biotype is {biotype}')
                    expected_length = 0
                    length = adata.raw.var.loc[e, 'feature_length']
                    result_list.append(length)
                    if length == expected_length:
                        print(f'feature_length {length} is the same as expected {expected_length}')
                        result_list.append(True)
                    else:
                        result_list.append(False)
            elif ('feature_biotype' in adata.raw.var.columns) and ('feature_length' not in adata.raw.var.columns):
                print('raw.var exists however feature_length is missing')
            else:
                print('raw.var exists however fields are missing')
        else:
            print('Checking adata.var')
            print('---------------------------')
            if ('feature_biotype' in adata.var.columns) and ('feature_length' in adata.var.columns):
                print('feature_biotype and feature_length are present in var fields')
                print('------------------------------------------------------------')
                print('Checking feature_length of: ', e)
                biotype = adata.var.loc[e, 'feature_biotype']
                result_list.append(biotype)
                if biotype == 'gene':
                    print('feature_biotype = gene')
                    length = adata.var.loc[e, 'feature_length']
                    result_list.append(length)
                    if length == expected_length:
                        print(f'feature_length {length} is the same as expected {expected_length}')
                        result_list.append(True)
                    else:
                        result_list.append(False)
                else:
                    print(f'biotype is {biotype}')
                    expected_length = 0
                    length = adata.var.loc[e, 'feature_length']
                    result_list.append(length)
                    if length == expected_length:
                        print(f'feature_length {length} is the same as expected {expected_length}')
                        result_list.append(True)
                    else:
                        result_list.append(False)
            elif ('feature_biotype' in adata.var.columns) and ('feature_length' not in adata.var.columns):
                print('feature_length is missing')
            else:
                print('fields are missing')
        result_dict[e] = result_list
    return pd.DataFrame(result_dict, columns=['ensg_id', 'expected_length','feature_biotype','feature_length','matching_lengths'])



In [48]:
# create example adatas of human, mouse and sars-cov-2 and save them to h5ads, subset the anndata objects
adata_human = sc.read_h5ad('../human_valid.h5ad') 
adata_mouse = sc.read_h5ad('../valid.h5ad') 


save(adata_human, 'human_valid_output.h5ad')
save(adata_mouse, 'mouse_valid_output.h5ad')


feature_biotype or feature_length is present
------------------
feature_length and feature_biotype are absent - CELLxGENE should annotate these two var fields
------------------


In [ ]:
adata_mouse.

In [ ]:
#validate--add-labels() to add var metadata to valid h5ads


In [ ]:
# create sample list of ensg_ids that we want to check (several from each organism)
ensg_ids = list()

In [ ]:
directory = '/Users/corinnsmall/GitClones/CZI/single-cell-curation/cellxgene_schema_cli/cellxgene_schema/ontology_files/'
list_of_gene_csvs = [directory + 'genes_homo_sapiens.csv.gz', directory + 'genes_mus_musculus.csv.gz', directory + 'genes_sars_cov_2.csv.gz', directory + 'genes_ercc_.csv.gz']

In [ ]:
# get results for each h5ad

output_files = ['../human_valid_output.h5ad','../mouse_valid_output.h5ad','../cov_valid_output.h5ad']
for file in output_files:
    result = check_var_metadata(file, ensg_ids, list_of_gene_csvs)

### Test info
- feature_length will be a new var field that the portal will add to each dataset upon submission: Annotator =	CELLxGENE Discover

- the --add-labels option for cellxgene-schema should be adding this too. 

- validation will be to run --add-labels to a .h5ad, then open the resulting .h5ad and do some checks:

    1. make sure var.feature_length is present

    2. if there’s a raw layer, make sure raw.var.feature_length is present

    3. then do some checks to make sure the lengths filled in are expected (we may need to pull some genes from v38 GENCODE gtf to use as checks here)

        - Lengths depend on feature_biotype = 'gene' or 'spike-in'
        - Gene lengths are calculated by creating non-overlapping concatenated exons across all isoforms of the gene, and then adding up their length in base-pairs.
        - Testing both var and raw.var

        <br> Valid cases:
        1. gene = summed length of non-overlapping concatenated exons across all isoforms of the gene
           
        2. spike-in = 0

        <br> Invalid cases:
        
        1. when feature_biotype = `gene` -> is gene length calculated as expected?
            - test when there are overlapping exons of various lengths (isoforms):
                - using gene found in v38 gencode gtf with multiple exons that vary in length
            
            - does it matter what strands exons are found on?
            
            - test null values?

            - test datatypes: str, float
            - any investigation into the use of cds vs exons?

        2. when feature_biotype = `spike-in` -> is length calculated as 0?
            - test non-zero value
            - test null value
            - test datatypes: str, float
            
